In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Lectura del archivo con la informacion a trabajar obtenida de Kaggle
df = pd.read_csv("../data/processed/df_final.csv")
df

,Venue,Result,Opponent,xG,xGA,Referee,Team
0,0,0,6,1.3,1.4,1,10
1,1,1,2,3.0,1.3,23,10
2,0,1,14,0.9,2.0,9,10
3,1,1,1,2.5,0.7,21,10
4,0,1,19,2.5,0.6,12,10
...,...,...,...,...,...,...,...
507,0,2,7,0.8,1.1,25,16
508,1,2,1,1.4,2.2,14,16
509,0,1,11,1.7,1.8,2,16
510,1,2,4,0.6,2.9,22,16


In [3]:
df['Result'].replace("D","0",inplace=True)
df['Result'].replace("W","1",inplace=True)
df['Result'].replace("L","2",inplace=True)


In [4]:
df.head(5)

,Venue,Result,Opponent,xG,xGA,Referee,Team
0,0,0,6,1.3,1.4,1,10
1,1,1,2,3.0,1.3,23,10
2,0,1,14,0.9,2.0,9,10
3,1,1,1,2.5,0.7,21,10
4,0,1,19,2.5,0.6,12,10


In [5]:
df.dtypes

Venue         int64
Result        int64
Opponent      int64
xG          float64
xGA         float64
Referee       int64
Team          int64
dtype: object

In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

categorical_cols = ['Venue', 'Opponent', 'Team']

label_encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = label_encoder.fit_transform(df[col])

X = df.drop('Result', axis=1)
y = df['Result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
#0:.Draw, 1:Win, 2:Lose 
df.head()

,Venue,Result,Opponent,xG,xGA,Referee,Team
0,0,0,6,1.3,1.4,1,10
1,1,1,2,3.0,1.3,23,10
2,0,1,14,0.9,2.0,9,10
3,1,1,1,2.5,0.7,21,10
4,0,1,19,2.5,0.6,12,10


In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Definir los hiperparámetros que deseas probar
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}

# Crear un modelo de regresión logística
logistic_regression_clf = LogisticRegression(random_state=42)

# Configurar la búsqueda de hiperparámetros usando GridSearchCV
grid_search = GridSearchCV(estimator=logistic_regression_clf, param_grid=param_grid, cv=5)

# Entrenar la búsqueda de hiperparámetros en el conjunto de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_
print("Mejores parámetros:", best_params)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The ma

Mejores parámetros: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [9]:
 #pip install mlflow
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

name_model = "LogisticRegression"
developer = "Angel De la mora, Regina Flores, Santiago Aguirre"

parameters = {
    "penalty": "l2",
    "C": 1.0,
    "solver": "lbfgs",
    "multi_class": "multinomial", 
    "max_iter": 100,
    "random_state": 42
}

with mlflow.start_run(run_name=name_model):
    
    mlflow.log_param("model", name_model)
    mlflow.log_param("developer", developer)
    
    # Registrar parámetros del modelo
    mlflow.log_params(parameters)
    
    # Entrenar un clasificador de regresión logística
    logistic_regression_clf = LogisticRegression(**parameters)
    logistic_regression_clf.fit(X_train, y_train)
    
    # Realizar predicciones
    predictions = logistic_regression_clf.predict(X_test)
    predictions_proba = logistic_regression_clf.predict_proba(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    roc_auc = roc_auc_score(y_test, predictions_proba, average='weighted', multi_class='ovr')

    # Registrar métricas
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(logistic_regression_clf, "logistic_regression_classifier")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
import dagshub
import mlflow

dagshub.init("PROYECTO-CD-Premiere-league", "IlseFlores", mlflow=True)

mlflow.set_tracking_uri('https://dag.com/IlseFlores/PROYECTO-CD-Premiere-league.mlflow')

Initialized MLflow to track repo "IlseFlores/PROYECTO-CD-Premiere-league"

Repository IlseFlores/PROYECTO-CD-Premiere-league initialized!

## Experimento1

In [68]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

name_model = "LogisticRegression"
developer = "Angel De la mora, Regina Flores, Santiago Aguirre"

parameters = {
    "penalty": "l2",
    "C": 1.0,
    "solver": "lbfgs",
    "multi_class": "multinomial", 
    "max_iter": 100,
    "random_state": 42
}

with mlflow.start_run(run_name=name_model):
    
    mlflow.log_param("model", name_model)
    mlflow.log_param("developer", developer)
    
    # Registrar parámetros del modelo
    mlflow.log_params(parameters)
    
    # Entrenar un clasificador de regresión logística
    logistic_regression_clf = LogisticRegression(**parameters)
    logistic_regression_clf.fit(X_train, y_train)
    
    # Realizar predicciones
    predictions = logistic_regression_clf.predict(X_test)
    predictions_proba = logistic_regression_clf.predict_proba(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    roc_auc = roc_auc_score(y_test, predictions_proba, average='weighted', multi_class='ovr')

    # Registrar métricas
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(logistic_regression_clf, "logistic_regression_classifier")


MlflowException: API request to https://dag.com/IlseFlores/PROYECTO-CD-Premiere-league.mlflow/api/2.0/mlflow/runs/create failed with exception HTTPSConnectionPool(host='dag.com', port=443): Max retries exceeded with url: /IlseFlores/PROYECTO-CD-Premiere-league.mlflow/api/2.0/mlflow/runs/create (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x3064b6550>: Failed to resolve 'dag.com' ([Errno 8] nodename nor servname provided, or not known)"))

## Experimento 2

In [69]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

param_grid = {
    'hidden_layer_sizes': [(100,), (50, 50), (50, 100, 50)],
    'activation': ['logistic', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive'],
}

mlp_clf = MLPClassifier(random_state=42)

grid_search = GridSearchCV(estimator=mlp_clf, param_grid=param_grid, cv=5)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Mejores parámetros:", best_params)


Mejores parámetros: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'solver': 'sgd'}


In [70]:
import mlflow
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

name_model = "MLPClassifier"
developer = "Angel De la mora, Regina Flores, Santiago Aguirre"

# Mejores parámetros encontrados
best_params = {
    "hidden_layer_sizes": (50, 50),
    "activation": "logistic",
    "solver": "adam",
    "alpha": 0.0001,
    "learning_rate": "constant",
    "max_iter": 1000,  # Aumentamos el número de iteraciones
    "random_state": 42
}

with mlflow.start_run(run_name=name_model):
    
    mlflow.log_param("model", name_model)
    mlflow.log_param("developer", developer)
    
    # Registrar parámetros del modelo
    mlflow.log_params(best_params)
    
    # Entrenar un clasificador de red neuronal
    mlp_clf = MLPClassifier(**best_params)
    mlp_clf.fit(X_train, y_train)
    
    # Realizar predicciones
    predictions = mlp_clf.predict(X_test)
    predictions_proba = mlp_clf.predict_proba(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    roc_auc = roc_auc_score(y_test, predictions_proba, average='weighted')

    # Registrar métricas
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(mlp_clf, "mlp_classifier")


MlflowException: API request to https://dag.com/IlseFlores/PROYECTO-CD-Premiere-league.mlflow/api/2.0/mlflow/runs/create failed with exception HTTPSConnectionPool(host='dag.com', port=443): Max retries exceeded with url: /IlseFlores/PROYECTO-CD-Premiere-league.mlflow/api/2.0/mlflow/runs/create (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x30c3343d0>: Failed to resolve 'dag.com' ([Errno 8] nodename nor servname provided, or not known)"))

### Experimento 3


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 300],  
    'max_depth': [None, 10, 20, 30],  
    'min_samples_split': [2, 5, 10],   
    'min_samples_leaf': [1, 2, 4],  
    
    'max_features': ['auto', 'sqrt'],  
}

rf_clf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=5)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Mejores parámetros:", best_params)

In [ ]:
import mlflow
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

name_model = "RandomForestClassifier"
developer = "Angel De la mora, Regina Flores, Santiago Aguirre"

# Mejores parámetros encontrados
best_params = {
    'max_depth': None,
    'max_features': 'sqrt',
    'min_samples_leaf': 1,
    'min_samples_split': 5,
    'n_estimators': 200,
    'random_state': 42
}

with mlflow.start_run(run_name=name_model):
    
    mlflow.log_param("model", name_model)
    mlflow.log_param("developer", developer)
    
    # Registrar parámetros del modelo
    mlflow.log_params(best_params)
    
    # Entrenar un clasificador de Random Forest
    rf_clf = RandomForestClassifier(**best_params)
    rf_clf.fit(X_train, y_train)
    
    # Realizar predicciones
    predictions = rf_clf.predict(X_test)
    predictions_proba = rf_clf.predict_proba(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    roc_auc = roc_auc_score(y_test, predictions_proba, average='weighted', multi_class='ovr')

    # Registrar métricas
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(rf_clf, "random_forest_classifier")